# Evaluation
This notebook has everything you'll need to perform the evaluation of the different models presented in the report.

In [5]:
# Imports
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from keras_preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
import albumentations as A
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
import cv2
import os
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout,AveragePooling2D
from tensorflow.keras.applications.resnet import ResNet50
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3,DenseNet201,EfficientNetB7, MobileNetV2,Xception,VGG16,NASNetMobile
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.models import Sequential
from keras.regularizers import *
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [6]:
# Preprocessing function
def preprocess(image):
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    im = cv2.filter2D(image, -1, kernel)
    out1=(cv2.normalize(im, (224,224),0, 255, cv2.NORM_MINMAX))
    return out1

In [7]:
# F1 score
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [8]:
# Read dataset and split it in train and validation using the strat label
train=pd.read_csv("content/phantom_images/Training_set.csv")
X_train, X_valid = train_test_split(train, test_size=0.1,stratify=train["stratlabel"],shuffle=True, random_state = 42)

In [9]:
# Create the Train generator
datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess)
train_generator=datagen.flow_from_dataframe(
dataframe=X_train,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 3873 validated image filenames belonging to 3 classes.


In [10]:
# Create the Validation generator
valid_datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess)
valid_generator=valid_datagen.flow_from_dataframe( 
dataframe=X_valid,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 431 validated image filenames belonging to 3 classes.


## Experiment 1:
Only last 3 neurons

In [34]:
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
for layer in conv_base.layers:
        layer.trainable = False
model.layers[-1].trainable = True
model.layers[-3].trainable = False
model.layers[-5].trainable = False
model.layers[-7].trainable = False

In [35]:
model.load_weights("training_0/phantoms_training_experiment1_seeded.weights.h5")

In [36]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.4485 - f1_score: 0.2988 - loss: 1.1093 - precision: 0.5118 - recall: 0.2114


In [37]:
print(outputs)

{'accuracy': 0.4570765793323517, 'f1_score': 0.312399297952652, 'loss': 1.1119531393051147, 'precision': 0.5105262994766235, 'recall': 0.22505800426006317}


## Experiment 2
8 and 3 neurons

In [38]:
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
for layer in conv_base.layers:
        layer.trainable = False
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = False
model.layers[-7].trainable = False

In [39]:
model.load_weights("training_0/phantoms_training_experiment2_seeded.weights.h5")

In [40]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.5774 - f1_score: 0.3934 - loss: 0.9783 - precision: 0.6982 - recall: 0.2742


In [41]:
print(outputs)

{'accuracy': 0.596287727355957, 'f1_score': 0.43934422731399536, 'loss': 0.9541032910346985, 'precision': 0.748603343963623, 'recall': 0.310904860496521}


## Experiment 3
All dense layers

In [42]:
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
for layer in conv_base.layers:
        layer.trainable = False
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True
model.layers[-7].trainable = False

In [43]:
model.load_weights("training_0/phantoms_training_experiment3_seeded.weights.h5")

In [44]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.7785 - f1_score: 0.7892 - loss: 0.9383 - precision: 0.8371 - recall: 0.7466


In [45]:
print(outputs)

{'accuracy': 0.7749419808387756, 'f1_score': 0.7852759957313538, 'loss': 0.947235643863678, 'precision': 0.8333333134651184, 'recall': 0.7424594163894653}


# Experiment 4
Same training done in the mammography repo but with the new dataset

In [46]:
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [47]:
model.load_weights("training_0/phantoms_training_experiment4_seeded.weights.h5")

In [48]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.8302 - f1_score: 0.8295 - loss: 0.7514 - precision: 0.8435 - recall: 0.8159


In [49]:
print(outputs)

{'accuracy': 0.8259860873222351, 'f1_score': 0.8268551230430603, 'loss': 0.7500692009925842, 'precision': 0.839712917804718, 'recall': 0.814385175704956}


# Experimento 5
More layers

In [50]:
model = Sequential()

conv_base = ResNet50(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:3]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)

model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [51]:
model.load_weights("training_0/phantoms_training_experiment5_seeded.weights.h5")

In [52]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 705ms/step - accuracy: 0.7769 - f1_score: 0.7871 - loss: 0.8946 - precision: 0.8093 - recall: 0.7662


In [53]:
print(outputs)

{'accuracy': 0.7865429520606995, 'f1_score': 0.7895362377166748, 'loss': 0.8945914506912231, 'precision': 0.8097561001777649, 'recall': 0.7703016400337219}


# Experiment 6
Resnet

In [54]:
model = Sequential()

conv_base = ResNet50(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)

model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [55]:
model.load_weights("training_0/phantoms_training_experiment6_seeded.weights.h5")

In [56]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 689ms/step - accuracy: 0.7546 - f1_score: 0.7547 - loss: 0.8962 - precision: 0.7702 - recall: 0.7399


In [57]:
print(outputs)

{'accuracy': 0.7772621512413025, 'f1_score': 0.7719714045524597, 'loss': 0.8876537680625916, 'precision': 0.7907542586326599, 'recall': 0.7540603280067444}


# Experiment 7
Inception

In [58]:
model = Sequential()

conv_base = InceptionV3(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)

model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [59]:
model.load_weights("training_0/phantoms_training_experiment7_seeded.weights.h5")

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 402 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [60]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 491ms/step - accuracy: 0.7856 - f1_score: 0.7889 - loss: 0.8517 - precision: 0.8039 - recall: 0.7746


In [61]:
print(outputs)

{'accuracy': 0.7865429520606995, 'f1_score': 0.7919620871543884, 'loss': 0.8408021330833435, 'precision': 0.8072289228439331, 'recall': 0.7772621512413025}


# Experiment 8
Change of learning rate

In [62]:
model = Sequential()

conv_base = InceptionV3(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/phantoms_training_experiment7_seeded.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [63]:
model.load_weights("training_0/phantoms_training_experiment8_seeded.weights.h5")

In [64]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 499ms/step - accuracy: 0.7791 - f1_score: 0.7871 - loss: 0.7390 - precision: 0.8359 - recall: 0.7437


In [66]:
print(outputs)

{'accuracy': 0.7888631224632263, 'f1_score': 0.7892813682556152, 'loss': 0.7509046196937561, 'precision': 0.8307692408561707, 'recall': 0.7517401576042175}


# Experiment 9
No label smoothing

In [67]:
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [68]:
model.load_weights("training_0/phantoms_training_experiment9_seeded.weights.h5")

In [69]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.7896 - f1_score: 0.7891 - loss: 0.8507 - precision: 0.7954 - recall: 0.7830


In [70]:
print(outputs)

{'accuracy': 0.814385175704956, 'f1_score': 0.8154204487800598, 'loss': 0.7576074600219727, 'precision': 0.8211764693260193, 'recall': 0.8097447752952576}


# Experiment 10
Data augmentation

In [71]:
datagen_aug=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess,vertical_flip=True)
train_generator_aug=datagen_aug.flow_from_dataframe(
dataframe=X_train,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 3873 validated image filenames belonging to 3 classes.


In [72]:
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [73]:
model.load_weights("training_0/phantoms_training_experiment10_seeded.weights.h5")

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1230 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [74]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.8306 - f1_score: 0.8221 - loss: 0.7174 - precision: 0.8348 - recall: 0.8099


In [75]:
print(outputs)

{'accuracy': 0.8352668285369873, 'f1_score': 0.8240849375724792, 'loss': 0.7321200966835022, 'precision': 0.838942289352417, 'recall': 0.8097447752952576}


# Experiment 11
More weight decay to the best model

In [76]:
model = Sequential()
conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [77]:
model.load_weights("training_0/phantoms_training_experiment11_seeded.weights.h5")

In [78]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.8355 - f1_score: 0.8346 - loss: 0.7762 - precision: 0.8508 - recall: 0.8190


In [79]:
print(outputs)

{'accuracy': 0.8236659169197083, 'f1_score': 0.8236685991287231, 'loss': 0.8222232460975647, 'precision': 0.8405796885490417, 'recall': 0.8074246048927307}


# Experiment 12
More augmentation

In [80]:
datagen_aug=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess,
                               vertical_flip=True, 
                               horizontal_flip = True,
                              zoom_range = 0.2)
train_generator_aug=datagen_aug.flow_from_dataframe(
dataframe=X_train,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 3873 validated image filenames belonging to 3 classes.


In [81]:
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [82]:
model.load_weights("training_0/phantoms_training_experiment12_seeded.weights.h5")

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1230 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [83]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.8451 - f1_score: 0.8427 - loss: 0.8035 - precision: 0.8445 - recall: 0.8410


In [84]:
print(outputs)

{'accuracy': 0.8468677401542664, 'f1_score': 0.844186007976532, 'loss': 0.775138258934021, 'precision': 0.8461538553237915, 'recall': 0.8422273993492126}


In [85]:
model.load_weights("training_0/exp12_best.weights.h5")

In [86]:
outputs = model.evaluate(valid_generator, return_dict = True)

14/14 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8514 - f1_score: 0.8432 - loss: 0.8067 - precision: 0.8538 - recall: 0.8329


In [87]:
print(outputs)

{'accuracy': 0.8677493929862976, 'f1_score': 0.8604923486709595, 'loss': 0.7536768913269043, 'precision': 0.8696682453155518, 'recall': 0.8515081405639648}
